In [67]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

In [68]:
interpreter = tf.lite.Interpreter(model_path="lite-model_movenet_singlepose_lightning_3.tflite")
interpreter.allocate_tensors()

In [69]:
lowerjoints_list = [13,14,15,16]

In [70]:
def locating_joints(frame, confidence_threshold):
    #y, x, c = frame.shape
    for i in lowerjoints_list:
        y, x, conf = joints_coordinates[i]
        if conf > confidence_threshold:
            cv2.circle(frame, (int(x), int(y)), 4, (0,255,0), -1)

In [71]:
def knee_distance(frame):
   # y, x, c = frame.shape
    y1,x1 ,_ = joints_coordinates[13]
    y2, x2,_ = joints_coordinates[14]
    import math
    temp = (x2-x1)**2+(y2-y1)**2
    distance = math.sqrt(temp)
    cv2.putText(frame,"Distance:" + str(distance), (0,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
    
    

In [72]:
def left_slope(frame):
  import math
  #y, x, c = frame.shape
  y1,x1 ,_ = joints_coordinates[15]
  y2, x2,_ = joints_coordinates[13]
  m = (y2-y1)/(x2-x1)
  cv2.putText(frame,"left slope:" + str(m), (50,100), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  

In [73]:
def ankle_distance(frame):
    #y, x, c = frame.shape
    y1,x1 ,_ = joints_coordinates[15]
    y2, x2,_ = joints_coordinates[16]
    import math
    temp = (x2-x1)**2+(y2-y1)**2
    distance = math.sqrt(temp)
    cv2.putText(frame," Ank:" + str(distance), (150,200), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
    
    

In [74]:
def right_slope(frame):
  import math
  #y, x, c = frame.shape
  y1,x1 ,_ = joints_coordinates[16]
  y2,x2,_ = joints_coordinates[14]
  m2 = (y2-y1)/(x2-x1)
  cv2.putText(frame,"Right slope:" + str(m2), (100,150), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  

In [75]:
EDGES = {
    (13, 15): 'm',
    (14, 16): 'm',
    (13, 14): 'm',
    (16,15) : 'm'
}

In [76]:
def join_joints(frame, keypoints, edges, confidence_threshold):
    #y, x, c = frame.shape
    print(keypoints.shape)

    for edge, color in edges.items():
        e1, e2 = edge
        y1, x1, c1 = joints_coordinates[e1]
        y2, x2, c2 = joints_coordinates[e2]
        
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
           cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)


In [77]:
def insoles(frame): 
  y1,x1 ,_ = joints_coordinates[13]
  y2,x2,_ = joints_coordinates[14] 
  y3,x3 ,_ = joints_coordinates[15]
  y4,x4,_ = joints_coordinates[16]
  import math 
  dis = (x2-x1)**2+(y2-y1)**2
  ankle_dis = (x4-x3)**2+(y4-y3)**2
  right_m= (y4-y2)/(x4-x2)
  left_m = (y3-y1)/(x3-x1)
  print("knee:", dis)
  print("ankle:", ankle_dis)
  print("left",left_m)
  print("Right",right_m)
#8739.635093408378
  
  #cv2.putText(frame,"insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  ankle = (x4-x3)**2+(y4-y3)**2
  #dis below 88 and ankle below 88, dis >80 and ankle > 80
  if (dis <= 7195.855191894665 and dis > 5608.360385571377 and ankle_dis <= 7195.855191894665 and ankle_dis > 5608.855051436771) or (dis < 7195.855191894665 and dis > 6495.855191894665 and ankle_dis >= 7195.855191894665 and ankle_dis <= 7795.855191894665 ):
     print("running loop 1")
     cv2.putText(frame,"Medium arch height insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  elif (dis <= 5608.855051436771 and ankle_dis <= 5608.855051436771) or (dis < 5608.855051436771 and ankle_dis > 5608.855051436771 and ankle_dis <= 5908.855051436771):
     #or (dis > 5608.855051436771 and ankle_dis < 7195.855191894665)
     print("running")
     cv2.putText(frame,"Low arch height insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  elif (dis >= 7195.855191894665 and ankle_dis >= 7195.855191894665):
     cv2.putText(frame,"High arch height insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  else:
     cv2.putText(frame,"Other medical condition", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
     pass
            
       
        
        

        
    
    

In [78]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0),192,192)
    input_image = tf.cast(img, dtype=tf.float32)
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    print(output_details)
    # predicting joints
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    print("hello boy")
    print(keypoints_with_scores)                
    y, x, c = frame.shape
    print("gurs")
    print(np.multiply(keypoints_with_scores, [y,x,1]))
    print("Kaur")
    joints_coordinates = np.squeeze(np.multiply(keypoints_with_scores, [y,x,1]))
    print(joints_coordinates)
    
    locating_joints(frame, 0.4) 
    join_joints(frame, keypoints_with_scores, EDGES, 0.4)
    knee_distance(frame)
    ankle_distance(frame)
    left_slope(frame)
    right_slope(frame)
    insoles(frame)
    cv2.imshow('MoveNet', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

[{'name': 'StatefulPartitionedCall:0', 'index': 312, 'shape': array([ 1,  1, 17,  3]), 'shape_signature': array([ 1,  1, 17,  3]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
hello boy
[[[[0.6198412  0.5703145  0.7142372 ]
   [0.5471431  0.61923945 0.69930655]
   [0.5594476  0.5117736  0.6942079 ]
   [0.55358046 0.676474   0.71047777]
   [0.5747243  0.4495129  0.7335539 ]
   [0.8032963  0.84303045 0.4870836 ]
   [0.7888397  0.31021917 0.64339626]
   [0.959465   0.93228114 0.04100385]
   [0.93709886 0.27664766 0.01647294]
   [0.81992286 0.75114334 0.0195246 ]
   [0.8537736  0.4497723  0.0107474 ]
   [1.0285366  0.8436448  0.00916362]
   [0.4050427  0.4406254  0.01554033]
   [0.86620355 0.97920984 0.01259512]
   [0.7979026  0.3377838  0.00527433]
   [0.82569975 0.83098316 0.00259942]
   [0.82304424 0.43311268 0.00220

In [66]:
cap.release()
cv2.destroyAllWindows()